In [9]:
import itertools
import numpy as np


def generate_states():
    storage_state = ["x","r","b","w"] #0 = empty, r = red, b = blue, w = white
    actions = ["sb", "sr", "sw", "rb", "rr", "rw"] #sb/sr/sw = store_blue/red/white || rb/rr/rw = restore blue/red/white
    #storage_states = [p for p in itertools.product(storage_state, repeat=4)]
    storage_states = []
    for action in actions:
        for x1 in storage_state:
            for x2 in storage_state:
                for x3 in storage_state:
                    for x4 in storage_state:
                        st = (x1, x2, x3, x4, action)
                        storage_states.append(st)

    print(len(storage_states))
    print(storage_states)
    return storage_states

In [10]:
#Matrix = [[0 for x in range(len(storage_states))] for y in range(len(storage_states))]
#print(len(Matrix[0]))
#print(len(Matrix[1535]))

In [144]:
import csv
import pandas as pd
def load_training_df():
    data = pd.read_csv("SAKI_Exercise_3_warehousetraining2x2.txt", delimiter='\t', names=["action", "color"])
    data.head()
    df = data.copy()
    return df

def load_test_df():
    data = pd.read_csv("SAKI_Exercise_3_warehouseorder2x2.txt", delimiter='\t', names=["action", "color"])
    data.head()
    df = data.copy()
    return df

In [145]:
def calc_container_distribution(dataframe):
    count_of_trainingset = len(dataframe)
    distribution_df = dataframe.copy()
    print(count_of_trainingset)
    distribution_df = distribution_df.groupby(['action', 'color']).size().reset_index(name='count')
    distribution_df['count'] = distribution_df['count'].div(count_of_trainingset)
    return distribution_df

In [146]:
def get_color(state):
    return state[4][1].lower()

def get_actioncolor(state):
    return state[4].lower()

def has_warehouse_free_slots(state):
    for s in state:
        if s == "x":
            return True
    return False

def does_warehouse_store_color(state, color_to_restore):
    for s in state:
        if s == color_to_restore:
            return True
    return False


def is_warehouse_empty(state):
    for s in state:
        if s != 'x':
            return False
    return True

def distribute_transition_probability_matrix(tpm):
    for index, row_vector in enumerate(tpm):
        #print(index, row_vector)
        sum = np.sum(row_vector)
        if sum == 0:
            #print('error 0')
            # no transition was possible for this state (e. g. restore from empty warehouse)
            # it should stay in the state then since the sum of each row has to equal one
            tpm[index, index] = 1
            continue
        #for j, obj in enumerate(row_vector):
            #print(j, obj)
    # give every possible transition the same possibility by dividing every element by the sum
    # of the row. E. g. [0, 0, 1, 1, 0, 0] will be converted to [0, 0, 0.5, 0.5, 0, 0].
    # This will also result in the sum of the row equal to one.
    #tpm = tpm / tpm.sum(axis=1)[:, None]
    return tpm

def is_store_action_possible(pos, state_1, state_2):
    if not has_warehouse_free_slots(state_1):
        return False

    color_to_store = get_color(state_1)

    for i in range(len(state_1)):
        if i == 4:
            break
        if i == pos:
            continue
        if state_1[i] != state_2[i]:
            return False
            
    if state_1[pos] == "x" and state_2[pos] != color_to_store and state_2[pos] != "x":
        # an empty slot was overwritten, but with something else we expected, not possible
        return False

    if state_1[pos] == "x" and state_2[pos] == color_to_store:
        return True
    
    return False


def is_restore_action_possible(pos, state_1, state_2):

    color_to_restore = get_color(state_1)
    if does_warehouse_store_color(state_1, color_to_restore) == False or is_warehouse_empty(state_1):
        # Warehouse does not store color, so we obviously can't restore it
        return False

    for i in range(len(state_1)):
        if i == 4:
            break
            
        if i == pos:
            continue
            
        if state_1[i] != state_2[i]:
            return False
            
    if state_1[pos] == "x":
        # an empty slot was overwritten, but with something else we expected, not possible
        return False
    
    if state_1[pos] == color_to_restore and state_2[pos] != "x" and state_2[pos] != color_to_restore:
        # an empty slot was overwritten, but with something else we expected, not possible
        return False

    if state_1[pos] == color_to_restore and state_2[pos] == 'x':
        return True
    
    return False

def is_transition_possible(pos, curr_state, next_state):
    curr_action = curr_state[4]
    if curr_action == "sb" or curr_action == "sr" or curr_action == "sw":
        return is_store_action_possible(pos, curr_state, next_state)
    else:
        return is_restore_action_possible(pos, curr_state, next_state)
    
def get_transition_probability_matrix(states):
    tp_matrix = np.zeros((len(states), len(states)), dtype=float, order='C')
    for x, curr_state in enumerate(states, start=0):
        for y, next_state in enumerate(states, start=0):
            if is_transition_possible(curr_state, next_state):
                tp_matrix[x, y] = 1
    return distribute_transition_probability_matrix(tp_matrix)


def create_tpm(states, distributions):
    tpm_all = []
    for pos in positions:
        tpm = np.zeros((len(states), len(states)),dtype=np.float16)
        for x, curr_state in enumerate(states, start=0):
            for y, next_state in enumerate(states, start=0):
                if is_transition_possible(pos, curr_state, next_state):
                    state_col = get_actioncolor(next_state)
                    if state_col == 'sr':
                        tpm[x, y] = distribution_df.loc[(distribution_df['action'] == 'store') & (distribution_df['color'] == 'red'), 'count'].item()
                    elif state_col == 'sw':
                        tpm[x, y] = distribution_df.loc[(distribution_df['action'] == 'store') & (distribution_df['color'] == 'white'), 'count'].item()
                    elif state_col == 'sb':
                        tpm[x, y] = distribution_df.loc[(distribution_df['action'] == 'store') & (distribution_df['color'] == 'blue'), 'count'].item()
                    elif state_col == 'rr':
                        tpm[x, y] = distribution_df.loc[(distribution_df['action'] == 'restore') & (distribution_df['color'] == 'red'), 'count'].item()
                    elif state_col == 'rw':
                        tpm[x, y] = distribution_df.loc[(distribution_df['action'] == 'restore') & (distribution_df['color'] == 'white'), 'count'].item()
                    else:
                        tpm[x, y] = distribution_df.loc[(distribution_df['action'] == 'restore') & (distribution_df['color'] == 'blue'), 'count'].item()
                    # switch case for all coloractions possible and then add percentage
                   
                  
        distributed_tpm = distribute_transition_probability_matrix(tpm)
        tpm_all.append(distributed_tpm)
    
    return tpm_all

In [147]:
def get_costs(index):
    costs_map = {
        0: 1,
        1: 2,
        2: 2,
        3: 3
    }
    return costs_map.get(index, 0)


def is_state_position_empty(state, pos):
    if state[pos] == 'x':
        return True
    return False

def get_reward_matrix(states, positions):
    #action_rm = np.zeros((1,len(states)))
    rm = np.zeros((len(states), len(positions)))
    print(rm)
    for pos in positions:
        #print(i, pos)
        for j, state in enumerate(states, start=0):
            if is_state_position_empty(state, pos):
                #print(pos, j)
                rm[j, pos] = pos+1
                
    print(rm)
    return rm
    

In [148]:
states = generate_states()
trainingsfile_df = load_training_df()
distribution_df = calc_container_distribution(trainingsfile_df)
test1 = distribution_df.loc[(distribution_df['action'] == 'store') & (distribution_df['color'] == 'red'), 'count'].item()
print(test1)
positions = [0,1,2,3]
tpm = create_tpm(states, distribution_df)
print(distribution_df)
#print(tpm)
reward_matrix = get_reward_matrix(states, positions)

1536
[('x', 'x', 'x', 'x', 'sb'), ('x', 'x', 'x', 'r', 'sb'), ('x', 'x', 'x', 'b', 'sb'), ('x', 'x', 'x', 'w', 'sb'), ('x', 'x', 'r', 'x', 'sb'), ('x', 'x', 'r', 'r', 'sb'), ('x', 'x', 'r', 'b', 'sb'), ('x', 'x', 'r', 'w', 'sb'), ('x', 'x', 'b', 'x', 'sb'), ('x', 'x', 'b', 'r', 'sb'), ('x', 'x', 'b', 'b', 'sb'), ('x', 'x', 'b', 'w', 'sb'), ('x', 'x', 'w', 'x', 'sb'), ('x', 'x', 'w', 'r', 'sb'), ('x', 'x', 'w', 'b', 'sb'), ('x', 'x', 'w', 'w', 'sb'), ('x', 'r', 'x', 'x', 'sb'), ('x', 'r', 'x', 'r', 'sb'), ('x', 'r', 'x', 'b', 'sb'), ('x', 'r', 'x', 'w', 'sb'), ('x', 'r', 'r', 'x', 'sb'), ('x', 'r', 'r', 'r', 'sb'), ('x', 'r', 'r', 'b', 'sb'), ('x', 'r', 'r', 'w', 'sb'), ('x', 'r', 'b', 'x', 'sb'), ('x', 'r', 'b', 'r', 'sb'), ('x', 'r', 'b', 'b', 'sb'), ('x', 'r', 'b', 'w', 'sb'), ('x', 'r', 'w', 'x', 'sb'), ('x', 'r', 'w', 'r', 'sb'), ('x', 'r', 'w', 'b', 'sb'), ('x', 'r', 'w', 'w', 'sb'), ('x', 'b', 'x', 'x', 'sb'), ('x', 'b', 'x', 'r', 'sb'), ('x', 'b', 'x', 'b', 'sb'), ('x', 'b', 'x'

In [224]:
def determine_new_warehouse(warehouse, pos, action, color):
    col_symb = determine_col_symbol(color)
    if action == 'store' and warehouse[pos] == 'x':
        warehouse[pos] = col_symb
    elif action == 'restore' and warehouse[pos] != 'x':
        warehouse[pos] = 'x'
    return warehouse
        
def determine_col_symbol(color):
    col_symb = ""
    if color == 'red':
        col_symb='r'
    if color == 'white':
        col_symb='w'
    if color == 'blue':
        col_symb='b'
    return col_symb

def get_warehouse_pos(policy, state):
    return policy[state]
    
def get_state_nr(warehouse):
    for index, state in enumerate(states, start=0):
        #print("--------------warehouse:---------------")
        #print(tuple(warehouse))
        #print("------<<--------state:---------<<------")
        #print(state)
        if state == tuple(warehouse):
            return index
    return False

def get_warehouse_state(warehouse, action, color):
    if action == 'store':
        if color == 'red':
            action='sr'
        if color == 'white':
            action='sw'
        if color == 'blue':
            action='sb'
    elif action == 'restore':
        if color == 'red':
            action='rr'
        if color == 'white':
            action='rw'
        if color == 'blue':
            action='rb'
    if len(warehouse) == 4:
        warehouse.append(action)
    elif len(warehouse) == 5:
        warehouse[4] = action
    return warehouse
        
def test_algorithm(policy: tuple):
    warehouse = ["x"]*len(positions)
    #print(warehouse)
    #print(type(warehouse))
    test_df = load_test_df()
    field_to_go = 0
    for index, row in test_df.iterrows():
        action_warehouse = get_warehouse_state(warehouse, row.action, row.color)
        state = get_state_nr(action_warehouse)
        if state != False:
            warehouse_position = get_warehouse_pos(policy, state)
            #print("warehouse_pos: ", warehouse_position)
            field_to_go += warehouse_position
            #print("field to go: ", field_to_go)
            warehouse = determine_new_warehouse(warehouse, warehouse_position, row.action, row.color)
            #print(warehouse)
        else:
            return 'failed'
    return field_to_go

def store_in_next_empty(warehouse, color):
    col_symb = determine_col_symbol(color)
    for index, pos in enumerate(warehouse, start=0):
        if pos == 'x':
            warehouse[index] = col_symb
            return index
        
    return False

def get_correct_first_container(warehouse, color):
    col_symb = determine_col_symbol(color)
    for index, pos in enumerate(warehouse, start=0):
        if pos == col_symb:
            warehouse[index] = "x"
            return index
        
    return False

def greedy_distance():
    warehouse = ["x"]*len(positions)
    test_df = load_test_df()
    distance_traveled = 0
    for index, row in test_df.iterrows():
        if row.action == 'store':
            distance_traveled += store_in_next_empty(warehouse, row.color)
            print(distance_traveled)
        elif row.action == 'restore':
            distance_traveled += get_correct_first_container(warehouse, row.color)
            print(distance_traveled)
    return distance_traveled

In [178]:
import mdptoolbox

mdpResultPolicy = mdptoolbox.mdp.PolicyIteration(tpm, reward_matrix, 0.8, max_iter=len(states) * 5)
mdpResultValue = mdptoolbox.mdp.ValueIteration(tpm, reward_matrix, 0.9, max_iter=len(states) * 5)
mdpRelativeValueIteration = mdptoolbox.mdp.RelativeValueIteration(tpm, reward_matrix, 0.9, max_iter=len(states)*5)
# Run the MDP
mdpResultPolicy.run()
mdpResultValue.run()

print('PolicyIteration:')
print(mdpResultPolicy.policy)
print(mdpResultPolicy.V)
print(mdpResultPolicy.iter)

print('ValueIteration:')
print(mdpResultValue.policy)
print(mdpResultValue.V)
print(mdpResultValue.iter)



PolicyIteration:
(2, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 3, 0, 0, 0, 3, 0, 0, 0, 3, 0, 0, 0, 2, 2, 2, 2, 3, 0, 0, 0, 3, 0, 0, 0, 3, 0, 0, 0, 2, 2, 2, 2, 3, 0, 0, 0, 3, 0, 0, 0, 3, 0, 0, 0, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 3, 0, 0, 0, 3, 0, 0, 0, 3, 0, 0, 0, 2, 2, 2, 2, 3, 0, 0, 0, 3, 0, 0, 0, 3, 0, 0, 0, 2, 2, 2, 2, 3, 0, 0, 0, 3, 0, 0, 0, 3, 0, 0, 0, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 3, 0, 0, 0, 3, 0, 0, 0, 3, 0, 0, 0, 2, 2, 2, 2, 3, 0, 0, 0, 3, 0, 0, 0, 3, 0, 0, 0, 2, 2, 2, 2, 3, 0, 0, 0, 3, 0, 0, 0, 3, 0, 0, 0, 2, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 1, 1, 1, 1, 1, 1

In [228]:
traveled_fields_mdpResultValue = test_algorithm(mdpResultValue.policy)
traveled_fields_mdpResultPolicy = test_algorithm(mdpResultPolicy.policy)
traveled_fields_greedy = greedy_distance()
print("ValueIteration Robot traveled: ", traveled_fields_mdpResultValue)
print("PolicyIteration Robot traveled: ", traveled_fields_mdpResultPolicy)
print("Greedy Robot traveled: ", traveled_fields_greedy)
#test_algorithm(mdpResultPolicy.policy)


0
1
2
3
5
8
8
9
12
12
14
14
14
14
14
15
17
19
19
19
21
23
23
23
25
28
29
32
32
32
32
32
33
36
37
38
39
42
42
44
44
45
47
48
48
50
50
51
53
56
57
57
59
62
62
62
62
63
65
68
70
70
71
71
72
ValueIteration Robot traveled:  95
PolicyIteration Robot traveled:  95
Greedy Robot traveled:  72
